In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [53]:
data = pd.read_excel("PQM.xlsx", sheet_name="Table1 (basic)", skiprows=[0])
data.head()

,HeatNo,Date,QualityRequirement,QualNo,CustID,CustVer,InternalVer,MetalRavneQualityName,SteelGroup,Month,...,Mo_Final,LFVD_FeMo,LFVD_Polymox,V_Last_EOP,V_Final,LFVD_FeV,W_Last_EOP,W_Final,LFVD_FeW72,LFVD_WPaketi
0,71601,2003-08-26 21:12:07,834.99.1 / 2,834,99,1,2,OCR12VM,Tool Steel; High Alloyed; 4 Cold,8,...,0.77,320.0,0.0,NaN,0.75,510.0,NaN,0.20,0.0,0.0
1,71602,2003-08-26 23:59:05,834.99.1 / 2,834,99,1,2,OCR12VM,Tool Steel; High Alloyed; 4 Cold,8,...,0.81,650.0,0.0,NaN,0.80,490.0,NaN,0.04,0.0,0.0
2,71609,2003-08-27 22:46:45,834.99.1 / 2,834,99,1,2,OCR12VM,Tool Steel; High Alloyed; 4 Cold,8,...,0.83,40.0,0.0,NaN,0.82,220.0,NaN,0.10,0.0,0.0
3,71610,2003-08-28 00:28:55,834.99.1 / 2,834,99,1,2,OCR12VM,Tool Steel; High Alloyed; 4 Cold,8,...,0.77,420.0,0.0,NaN,0.82,390.0,NaN,0.20,0.0,0.0
4,71616,2003-09-01 19:03:58,834.65.1 / 7,834,65,1,7,OCR12VM,Tool Steel; High Alloyed; 4 Cold,9,...,0.80,20.0,0.0,NaN,0.95,290.0,NaN,0.18,0.0,0.0


In [54]:
tables = ["Table1 (basic)", "Table2 (limits)", 
          "Table3 (events)", "Table4 (alloys)", 
          "Table5 (temps)", "Table6 (weights)", 
          "Table7 (head)", "Table8 (scrap)",
          "Table9 (all. scr.)", "Table10 (unall. scr.)"]

data = dict()

for i in range(len(tables)):
    data[i + 1] = pd.read_excel("PQM.xlsx", sheet_name=tables[i], skiprows=[0])
    tables[i] = tables[i].lower().replace(' ', '_').replace('(','').replace(')','')
    data[i + 1].to_csv(tables[i] + ".csv", sep=";", encoding="utf-8")


In [55]:
import psycopg2 as psycopg2

# Создаёт таблицу с именем table_name из датафрейма dataframe в базе данных из connection
def create_table_from_dataframe(connection, table_name, dataframe):
    query = "CREATE TABLE IF NOT EXISTS " + table_name + "(\n "

    data_types = {
        "int64": 'integer',
        "datetime64[ns]": 'timestamp',
        "object": 'text',
        "float64": 'real',
    }

    add_new_line_flag = False

    #print(dataframe.dtypes)
    dataframe_types = dataframe.dtypes

    for column in dataframe.columns:
        if add_new_line_flag:
            query += ",\n   "
        query += '"' + column + '" ' + data_types[str(dataframe_types[column])]
        add_new_line_flag = True

    query += ");"

    print(query)

    cursor = connection.cursor()
    cursor.execute(query)

    cursor.close()
    connection.commit()


def insert_data_from_csv(connection, table_name, csv_file_name):
    query = "COPY " + table_name + " FROM '" + csv_file_name + "' WITH (FORMAT CSV, DELIMITER ';', HEADER TRUE);"

    print(query)

    cursor = connection.cursor()
    cursor.execute(query)

    cursor.close()
    connection.commit()

In [56]:
# создаём подключение к бд
conn = psycopg2.connect(
    host="localhost",
    database="real_time_process",
    user="postgres",
    password="admin")

In [57]:
import os

dir_path = os.getcwd()

for i in range(len(tables)):
    create_table_from_dataframe(conn, tables[i], data[i + 1])
    insert_data_from_csv(conn, tables[i], os.path.join(dir_path, tables[i] + '.csv'))

CREATE TABLE IF NOT EXISTS table1_basic(
 "HeatNo" integer,
   "Date" timestamp,
   "QualityRequirement" text,
   "QualNo" integer,
   "CustID" integer,
   "CustVer" integer,
   "InternalVer" integer,
   "MetalRavneQualityName" text,
   "SteelGroup" text,
   "Month" integer,
   "Year" integer,
   "TotalIngotsWeight" real,
   "PouringScrap" real,
   "OtherScrap" real,
   "Last_EOP" real,
   "Cr_Last_EOP" real,
   "Cr_Final" real,
   "LFVD_FeCrA" real,
   "LFVD_FeCrC" real,
   "Ni_Last_EOP" real,
   "Ni_Final" real,
   "LFVD_NiGran" real,
   "LFVD_NiKatode" real,
   "Mo_Last_EOP" real,
   "Mo_Final" real,
   "LFVD_FeMo" real,
   "LFVD_Polymox" real,
   "V_Last_EOP" real,
   "V_Final" real,
   "LFVD_FeV" real,
   "W_Last_EOP" real,
   "W_Final" real,
   "LFVD_FeW72" real,
   "LFVD_WPaketi" real);
COPY table1_basic FROM 'c:\My information\MISiS\6 semestr\Real time system\table1_basic.csv' WITH (FORMAT CSV, DELIMITER ';', HEADER TRUE);


BadCopyFileFormat: ОШИБКА:  лишние данные после содержимого последнего столбца
CONTEXT:  COPY table1_basic, строка 2: "0;71601;2003-08-26 21:12:07;834.99.1 / 2;834;99;1;2;OCR12VM;"Tool Steel; High Alloyed; 4 Cold";8;200..."
